In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [2]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [3]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [4]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [11]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [13]:
num_epochs = 10
num_sims = 10
train_percs = [1.0,0.8,0.8]
synth_percs = [0.0,0.0,0.2]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = models.resnet18(weights='IMAGENET1K_V1')

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_resnet_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0031 Acc: 0.5213
test Loss: 1.0738 Acc: 0.4648
Epoch 1/9
----------
synthetic_train Loss: 1.0020 Acc: 0.5160
test Loss: 0.9286 Acc: 0.5602
Epoch 2/9
----------
synthetic_train Loss: 0.9624 Acc: 0.5437
test Loss: 0.9854 Acc: 0.5148
Epoch 3/9
----------
synthetic_train Loss: 0.9481 Acc: 0.5469
test Loss: 0.9503 Acc: 0.5250
Epoch 4/9
----------
synthetic_train Loss: 0.9285 Acc: 0.5537
test Loss: 0.8898 Acc: 0.5484
Epoch 5/9
----------
synthetic_train Loss: 0.8987 Acc: 0.5742
test Loss: 0.9868 Acc: 0.5148
Epoch 6/9
----------
synthetic_train Loss: 0.8781 Acc: 0.5908
test Loss: 0.8845 Acc: 0.5844
Epoch 7/9
----------
synthetic_train Loss: 0.7918 Acc: 0.6512
test Loss: 0.7052 Acc: 0.6828
Epoch 8/9
----------
synthetic_train Loss: 0.6504 Acc: 0.7316
test Loss: 0.5795 Acc: 0.7656
Epoch 9/9
----------
synthetic_train Loss: 0.4428 Acc: 0.8230
test Loss: 0.4331 Acc: 0.8344

Training complete in 8m 3s
Best test Loss: 0.433059


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.9872 Acc: 0.5340
test Loss: 0.9180 Acc: 0.5469
Epoch 1/9
----------
synthetic_train Loss: 0.9525 Acc: 0.5471
test Loss: 1.0245 Acc: 0.5102
Epoch 2/9
----------
synthetic_train Loss: 0.9146 Acc: 0.5711
test Loss: 0.9752 Acc: 0.4891
Epoch 3/9
----------
synthetic_train Loss: 0.8451 Acc: 0.6174
test Loss: 0.9003 Acc: 0.5687
Epoch 4/9
----------
synthetic_train Loss: 0.7596 Acc: 0.6664
test Loss: 0.7279 Acc: 0.6984
Epoch 5/9
----------
synthetic_train Loss: 0.5770 Acc: 0.7646
test Loss: 0.4578 Acc: 0.8148
Epoch 6/9
----------
synthetic_train Loss: 0.3656 Acc: 0.8682
test Loss: 0.2340 Acc: 0.9172
Epoch 7/9
----------
synthetic_train Loss: 0.1854 Acc: 0.9365
test Loss: 0.2094 Acc: 0.9180
Epoch 8/9
----------
synthetic_train Loss: 0.1285 Acc: 0.9539
test Loss: 0.1445 Acc: 0.9516
Epoch 9/9
----------
synthetic_train Loss: 0.0902 Acc: 0.9689
test Loss: 0.0878 Acc: 0.9680

Training complete in 8m 1s
Best test Loss: 0.087790


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 1.1086 Acc: 0.4703
test Loss: 1.0555 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0706 Acc: 0.4682
test Loss: 1.0476 Acc: 0.4953
Epoch 2/9
----------
synthetic_train Loss: 1.0563 Acc: 0.4781
test Loss: 1.0513 Acc: 0.4953
Epoch 3/9
----------
synthetic_train Loss: 1.0566 Acc: 0.4809
test Loss: 1.0540 Acc: 0.4953
Epoch 4/9
----------
synthetic_train Loss: 1.0449 Acc: 0.4881
test Loss: 1.0501 Acc: 0.4953
Epoch 5/9
----------
synthetic_train Loss: 1.0432 Acc: 0.4961
test Loss: 1.3284 Acc: 0.3586
Epoch 6/9
----------
synthetic_train Loss: 1.0412 Acc: 0.4926
test Loss: 1.0501 Acc: 0.4953
Epoch 7/9
----------
synthetic_train Loss: 1.0439 Acc: 0.4992
test Loss: 1.0870 Acc: 0.4953
Epoch 8/9
----------
synthetic_train Loss: 1.0435 Acc: 0.4967
test Loss: 1.0246 Acc: 0.5023
Epoch 9/9
----------
synthetic_train Loss: 1.0397 Acc: 0.4963
test Loss: 1.0491 Acc: 0.4953

Training complete in 8m 12s
Best test Loss: 1.024631


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0868 Acc: 0.4678
test Loss: 1.0838 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0740 Acc: 0.4715
test Loss: 1.6853 Acc: 0.4953
Epoch 2/9
----------
synthetic_train Loss: 1.0607 Acc: 0.4816
test Loss: 1.0571 Acc: 0.4953
Epoch 3/9
----------
synthetic_train Loss: 1.0525 Acc: 0.4879
test Loss: 1.0522 Acc: 0.4953
Epoch 4/9
----------
synthetic_train Loss: 1.0538 Acc: 0.4887
test Loss: 1.5136 Acc: 0.3586
Epoch 5/9
----------
synthetic_train Loss: 1.0461 Acc: 0.4914
test Loss: 1.0774 Acc: 0.5102
Epoch 6/9
----------
synthetic_train Loss: 1.0448 Acc: 0.4982
test Loss: 1.1610 Acc: 0.4953
Epoch 7/9
----------
synthetic_train Loss: 1.0408 Acc: 0.4998
test Loss: 1.0401 Acc: 0.4953
Epoch 8/9
----------
synthetic_train Loss: 1.0415 Acc: 0.4955
test Loss: 1.0475 Acc: 0.4953
Epoch 9/9
----------
synthetic_train Loss: 1.0390 Acc: 0.4996
test Loss: 1.0446 Acc: 0.4930

Training complete in 8m 20s
Best test Loss: 1.040102


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.1072 Acc: 0.4652
test Loss: 1.3513 Acc: 0.3555
Epoch 1/9
----------
synthetic_train Loss: 1.0679 Acc: 0.4787
test Loss: 1.0448 Acc: 0.4953
Epoch 2/9
----------
synthetic_train Loss: 1.0587 Acc: 0.4793
test Loss: 1.3622 Acc: 0.3156
Epoch 3/9
----------
synthetic_train Loss: 1.0501 Acc: 0.4895
test Loss: 1.4087 Acc: 0.4953
Epoch 4/9
----------
synthetic_train Loss: 1.0466 Acc: 0.4984
test Loss: 1.0472 Acc: 0.4953
Epoch 5/9
----------
synthetic_train Loss: 1.0439 Acc: 0.4941
test Loss: 1.0422 Acc: 0.4953
Epoch 6/9
----------
synthetic_train Loss: 1.0441 Acc: 0.4938
test Loss: 1.0505 Acc: 0.4953
Epoch 7/9
----------
synthetic_train Loss: 1.0423 Acc: 0.4994
test Loss: 1.0401 Acc: 0.4953
Epoch 8/9
----------
synthetic_train Loss: 1.0387 Acc: 0.5000
test Loss: 1.0431 Acc: 0.4953
Epoch 9/9
----------
synthetic_train Loss: 1.0393 Acc: 0.4973
test Loss: 1.0383 Acc: 0.4953

Training complete in 8m 32s
Best test Loss: 1.038316
